In [7]:
import requests
from bs4 import BeautifulSoup
import pandas
import time
import json
import sys

In [2]:
def get_last_page_num(url):
    response = requests.get(url)
    if response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, "html.parser")
        pagination = soup.find("ul", class_="pagination")
        last_page = pagination.find_all("li")[-2]
        last_page_num = int(last_page.text)
        return last_page_num

def get_app_lot_tiles(soup):
    lot_tiles = soup.find_all("app-lot-tile")

    auction_info_list = []

    # Loop through each lot tile
    for lot_tile in lot_tiles:
        lot_lead_heading = lot_tile.find("div", class_="lot-lead-heading")
        lot_number = lot_lead_heading.find("span", class_="text-primary").text.split(" ")[1] # might be of form '41769r'
        lot_title = lot_lead_heading.find("h2", class_="lot-title").text

        lot_high_bid_status = lot_tile.find("span", class_="lot-high-bid")
        lot_high_bid_text = lot_high_bid_status.find("span", class_="d-sm-inline").text
        lot_high_bid_parts = lot_high_bid_text.split(":")
        lot_high_bid_amount, lot_high_bid_currency = lot_high_bid_parts[1].strip().split(" ")
        lot_high_bid_amount = float(lot_high_bid_amount)

        lot_bid_history = lot_tile.find("app-bid-history-link")
        number_of_bids = int(lot_bid_history.find("a", class_="lot-bid-history").text.strip().split(" ")[0])

        auction_link = lot_lead_heading.find("a", class_="lot-title-ellipsis").get("href")

        auction_info = {
            "link": auction_link,
            "title": lot_title,
            "lot_number": lot_number,
            "number_of_bids": number_of_bids,
            "currency": lot_high_bid_currency,
            "max_bid": lot_high_bid_amount
        }

        auction_info_list.append(auction_info)

    return auction_info_list


def get_lots(url):
    lots = []
    adstring = "?apage="
    last_page_num = get_last_page_num(url)
    for page_num in range(1, last_page_num + 1):
        print(f"Getting data from page {page_num} of {last_page_num}")
        page_url = url + adstring + str(page_num)
        response = requests.get(page_url)
        if response.status_code == 200:
            # Parse the HTML content using BeautifulSoup
            soup = BeautifulSoup(response.content, "html.parser")

            # Get the app-lot-tile elements
            app_lot_tiles_data = get_app_lot_tiles(soup)
            lots.extend(app_lot_tiles_data)
    return lots

def save_to_csv(data, filename):
    df = pandas.DataFrame(data)
    df.to_csv(filename, index=False)
       

# url = "https://hibid.com/catalog/481347/new-and-returned-merchandise-online-auction--113"
# save_to_csv(get_lots(url), "test_auction_data.csv")

In [8]:

def get_lot_details(lot_id):
    graphql_url = "https://hibid.com/graphql" 

    headers = {
        "Content-Type": "application/json",
    }

    query = "query GetLotDetails($lotId: ID!, $countAsView: Boolean = true) {\n  lot(input: $lotId, countAsView: $countAsView) {\n    accessability\n    lot {\n      ...lotFull\n      __typename\n    }\n    __typename\n  }\n}\n\nfragment lotFull on Lot {\n  auction {\n    ...auction\n    __typename\n  }\n  ...lotOnly\n  __typename\n}\n\nfragment auction on Auction {\n  id\n  altBiddingUrl\n  altBiddingUrlCaption\n  amexAccepted\n  discoverAccepted\n  mastercardAccepted\n  visaAccepted\n  regType\n  holdAmount\n  termsAndConditions\n  auctioneer {\n    ...auctioneer\n    __typename\n  }\n  auctionNotice\n  auctionOptions {\n    bidding\n    altBidding\n    catalog\n    liveCatalog\n    shippingType\n    preview\n    registration\n    webcast\n    useLotNumber\n    useSaleOrder\n    __typename\n  }\n  auctionState {\n    auctionStatus\n    bidCardNumber\n    isRegistered\n    openLotCount\n    timeToOpen\n    __typename\n  }\n  bidAmountType\n  biddingNotice\n  bidIncrements {\n    minBidIncrement\n    upToAmount\n    __typename\n  }\n  bidType\n  buyerPremium\n  buyerPremiumRate\n  checkoutDateInfo\n  previewDateInfo\n  currencyAbbreviation\n  description\n  eventAddress\n  eventCity\n  eventDateBegin\n  eventDateEnd\n  eventDateInfo\n  eventName\n  eventState\n  eventZip\n  featuredPicture {\n    description\n    fullSizeLocation\n    height\n    thumbnailLocation\n    width\n    __typename\n  }\n  links {\n    description\n    id\n    type\n    url\n    videoId\n    __typename\n  }\n  lotCount\n  showBuyerPremium\n  audioVideoChatInfo {\n    aVCEnabled\n    blockChat\n    __typename\n  }\n  shippingAndPickupInfo\n  paymentInfo\n  hidden\n  sourceType\n  __typename\n}\n\nfragment auctioneer on Auctioneer {\n  address\n  bidIncrementDisclaimer\n  buyerRegNotesCaption\n  city\n  countryId\n  country\n  cRMID\n  currencyExpressUrl\n  email\n  fax\n  id\n  internetAddress\n  missingThumbnail\n  name\n  noMinimumCaption\n  phone\n  state\n  postalCode\n  __typename\n}\n\nfragment lotOnly on Lot {\n  bidAmount\n  bidList\n  bidQuantity\n  description\n  estimate\n  featuredPicture {\n    description\n    fullSizeLocation\n    height\n    thumbnailLocation\n    width\n    __typename\n  }\n  forceLiveCatalog\n  fr8StarUrl\n  hideLeadWithDescription\n  id\n  itemId\n  lead\n  links {\n    description\n    id\n    type\n    url\n    videoId\n    __typename\n  }\n  linkTypes\n  lotNavigator {\n    lotCount\n    lotPosition\n    nextId\n    previousId\n    __typename\n  }\n  lotNumber\n  lotState {\n    ...lotState\n    __typename\n  }\n  pictureCount\n  pictures {\n    description\n    fullSizeLocation\n    height\n    thumbnailLocation\n    width\n    __typename\n  }\n  quantity\n  ringNumber\n  rv\n  category {\n    baseCategoryId\n    categoryName\n    description\n    fullCategory\n    header\n    id\n    parentCategoryId\n    uRLPath\n    __typename\n  }\n  shippingOffered\n  simulcastStatus\n  site {\n    currencyExpressUrl\n    domain\n    fr8StarUrl\n    isDomainRequest\n    isExtraWWWRequest\n    siteType\n    subdomain\n    __typename\n  }\n  saleOrder\n  __typename\n}\n\nfragment lotState on LotState {\n  bidCount\n  biddingExtended\n  bidMax\n  bidMaxTotal\n  buyerBidStatus\n  buyerHighBid\n  buyerHighBidTotal\n  buyNow\n  choiceType\n  highBid\n  highBuyerId\n  isArchived\n  isClosed\n  isHidden\n  isLive\n  isNotYetLive\n  isOnLiveCatalog\n  isPosted\n  isPublicHidden\n  isRegistered\n  isWatching\n  linkedSoftClose\n  mayHaveWonStatus\n  minBid\n  priceRealized\n  priceRealizedMessage\n  priceRealizedPerEach\n  productStatus\n  productUrl\n  quantitySold\n  reserveSatisfied\n  sealed\n  showBidStatus\n  showReserveStatus\n  softCloseMinutes\n  softCloseSeconds\n  status\n  timeLeft\n  timeLeftLead\n  timeLeftSeconds\n  timeLeftTitle\n  timeLeftWithLimboSeconds\n  timeLeftWithLimboSeconds\n  watchNotes\n  __typename\n}" 
    
    payload = {
        "query": query,
        "variables": {
            "lotId": lot_id,
            "countAsView": True
        }
    }

    response = requests.post(graphql_url, headers=headers, json=payload)

    if response.status_code == 200:
        result = response.json()
        return result
    else:
        print("Error:", response.status_code)
        return None

lot_id = "168624517"  # Replace with the actual lot ID
lot_details = get_lot_details(lot_id)

# if lot_details:
#     # print lot details pretty
#     print(json.dumps(lot_details, indent=4, sort_keys=True))


In [5]:
def get_lots_from_live_auction(lot_id, get_time_left=False):
    graphql_url = "https://hibid.com/graphql" 

    query = """
        query LiveCatalogLots($auctionId: Int!) {
            liveCatalogLots(id: $auctionId) {
                auction {
                    ...auction
                    __typename
                }
                liveLots {
                    ...lotOnly
                    __typename
                }
                upcomingLots {
                    ...lotOnly
                    __typename
                }
                __typename
            }
        }

        fragment auction on Auction {
            id
            altBiddingUrl
            altBiddingUrlCaption
            amexAccepted
            discoverAccepted
            mastercardAccepted
            visaAccepted
            regType
            holdAmount
            termsAndConditions
            auctioneer {
                ...auctioneer
                __typename
            }
            auctionNotice
            auctionOptions {
                bidding
                altBidding
                catalog
                liveCatalog
                shippingType
                preview
                registration
                webcast
                useLotNumber
                useSaleOrder
                __typename
            }
            auctionState {
                auctionStatus
                bidCardNumber
                isRegistered
                openLotCount
                timeToOpen
                __typename
            }
            bidAmountType
            biddingNotice
            bidIncrements {
                minBidIncrement
                upToAmount
                __typename
            }
            bidType
            buyerPremium
            buyerPremiumRate
            checkoutDateInfo
            previewDateInfo
            currencyAbbreviation
            description
            eventAddress
            eventCity
            eventDateBegin
            eventDateEnd
            eventDateInfo
            eventName
            eventState
            eventZip
            featuredPicture {
                description
                fullSizeLocation
                height
                thumbnailLocation
                width
                __typename
            }
            links {
                description
                id
                type
                url
                videoId
                __typename
            }
            lotCount
            showBuyerPremium
            audioVideoChatInfo {
                aVCEnabled
                blockChat
                __typename
            }
            shippingAndPickupInfo
            paymentInfo
            hidden
            sourceType
            __typename
        }

        fragment auctioneer on Auctioneer {
            address
            bidIncrementDisclaimer
            buyerRegNotesCaption
            city
            countryId
            country
            cRMID
            currencyExpressUrl
            email
            fax
            id
            internetAddress
            missingThumbnail
            name
            noMinimumCaption
            phone
            state
            postalCode
            __typename
        }

        fragment lotOnly on Lot {
            bidAmount
            bidList
            bidQuantity
            description
            estimate
            featuredPicture {
                description
                fullSizeLocation
                height
                thumbnailLocation
                width
                __typename
            }
            forceLiveCatalog
            fr8StarUrl
            hideLeadWithDescription
            id
            itemId
            lead
            links {
                description
                id
                type
                url
                videoId
                __typename
            }
            linkTypes
            lotNavigator {
                lotCount
                lotPosition
                nextId
                previousId
                __typename
            }
            lotNumber
            lotState {
                ...lotState
                __typename
            }
            pictureCount
            pictures {
                description
                fullSizeLocation
                height
                thumbnailLocation
                width
                __typename
            }
            quantity
            ringNumber
            rv
            category {
                baseCategoryId
                categoryName
                description
                fullCategory
                header
                id
                parentCategoryId
                uRLPath
                __typename
            }
            shippingOffered
            simulcastStatus
            site {
                currencyExpressUrl
                domain
                fr8StarUrl
                isDomainRequest
                isExtraWWWRequest
                siteType
                subdomain
                __typename
            }
            saleOrder
            __typename
        }

        fragment lotState on LotState {
            bidCount
            biddingExtended
            bidMax
            bidMaxTotal
            buyerBidStatus
            buyerHighBid
            buyerHighBidTotal
            buyNow
            choiceType
            highBid
            highBuyerId
            isArchived
            isClosed
            isHidden
            isLive
            isNotYetLive
            isOnLiveCatalog
            isPosted
            isPublicHidden
            isRegistered
            isWatching
            linkedSoftClose
            mayHaveWonStatus
            minBid
            priceRealized
            priceRealizedMessage
            priceRealizedPerEach
            productStatus
            productUrl
            quantitySold
            reserveSatisfied
            sealed
            showBidStatus
            showReserveStatus
            softCloseMinutes
            softCloseSeconds
            status
            timeLeft
            timeLeftLead
            timeLeftSeconds
            timeLeftTitle
            timeLeftWithLimboSeconds
            timeLeftWithLimboSeconds
            watchNotes
            __typename
        }
    """
    
    payload = {
        "operationName": "LiveCatalogLots",
        "query": query,
        "variables": {
            "auctionId": lot_id
        }
    }

    response = requests.post(graphql_url, json=payload)
    if response.status_code == 200:
        result = response.json()
        if get_time_left:
            return result["data"]["liveCatalogLots"]["liveLots"][0]["lotState"]["timeLeftSeconds"]
        return result
    else:
        print("Error:", response.status_code)
        return None

get_lots_from_live_auction(480692)

# # save to json file
# with open("api_result.json", "w") as f:
#     json.dump(out, f, indent=4, sort_keys=True)

{'data': {'liveCatalogLots': {'auction': {'id': 480692,
    'altBiddingUrl': '',
    'altBiddingUrlCaption': '',
    'amexAccepted': False,
    'discoverAccepted': False,
    'mastercardAccepted': True,
    'visaAccepted': True,
    'regType': 'CREDIT_CARD_EVERY_TIME',
    'holdAmount': 0,
    'termsAndConditions': 'All items sold are of Returned Merchandise/Undeliverable Freight\xa0and are not inspected.\r\n\r\nAuction Terms & Conditions\r\n\r\nFEES:\r\n\r\nAll items will incur a 16% Buyers Premium, as well as HST and a $1.00 handling fee.\r\n\r\nPAYMENT:\r\n\r\nPayment for all items is mandatory by 2 pm the day following the auction. If you win the bid, we expect payment via credit card, and we reserve the right to charge your credit card for the total amount stated in the invoice. Credit cards are processed the morning after the auction, and a copy of the receipt will be sent to you by 10 am that day. If you decide to pay via e-transfer, debit, cash, or any other payment method, ple

In [5]:
def track_and_update_data(lot_id, file_name, sleep_time=60):
    num_tries = 0
    d = {
        "auction" : {},
        "lots":[]
    }
    while True:
        data = get_lots_from_live_auction(lot_id)
        if data:
            d["auction"] = data["data"]["liveCatalogLots"]["auction"]
            live_lots = data["data"]["liveCatalogLots"]["liveLots"]
            for lot in live_lots:
                if lot['itemId'] not in [i['itemId'] for i in d["lots"]]:
                    d["lots"].append(lot)
        elif num_tries > 5:
            break
        else:
            num_tries += 1
        # save to json file
        with open(file_name, "w") as f:
            json.dump(d, f, indent=4, sort_keys=True)
        time.sleep(sleep_time)


In [10]:
def get_scheduled_auctions(auctioneer_id, closest_auction_only=True):
  
  graphql_url = "https://hibid.com/graphql" 
  query = """query AuctionsByAuctioneerSearch($auctioneerId: Int, $status: AuctionLotStatus = null, $pageNumber: Int, $pageLength: Int) {
  auctionSearch(
    input: {auctioneerId: $auctioneerId, status: $status}
    pageNumber: $pageNumber
    pageLength: $pageLength
  ) {
    pagedResults {
      pageLength
      pageNumber
      totalCount
      filteredCount
      results {
        matchinglotcount
        matchingCategoryCounts {
          category
          count
          featuredPicture {
            fullSizeLocation
            thumbnailLocation
            description
            width
            height
            __typename
          }
          __typename
        }
        auction {
          ...AuctionHeader
          __typename
        }
        __typename
      }
      __typename
    }
    __typename
  }
  }

  fragment AuctionHeader on Auction {
  id
  altBiddingUrl
  altBiddingUrlCaption
  auctioneer {
    id
    name
    missingThumbnail
    __typename
  }
  auctionOptions {
    bidding
    altBidding
    catalog
    liveCatalog
    shippingType
    preview
    registration
    webcast
    useLotNumber
    useSaleOrder
    __typename
  }
  auctionState {
    auctionStatus
    isRegistered
    openLotCount
    timeToOpen
    __typename
  }
  description
  eventName
  eventAddress
  eventCity
  eventState
  eventZip
  eventDateBegin
  eventDateEnd
  eventDateInfo
  bidType
  biddingNotice
  auctionNotice
  links {
    description
    id
    type
    url
    videoId
    __typename
  }
  lotCount
  showBuyerPremium
  sourceType
  featuredPicture {
    description
    fullSizeLocation
    height
    thumbnailLocation
    width
    __typename
  }
  __typename
  }"""

  payload = {
    "operationName": "AuctionsByAuctioneerSearch",
    "query": query,
    "variables": {
      "auctioneerId": auctioneer_id,
      "pageLength": 25,
      "pageNumber": 1,
      "status": "OPEN"
    }
  }

  response = requests.post(graphql_url, json=payload)
  if response.status_code == 200:
    result = response.json()
    if closest_auction_only:
      auc_id = result["data"]["auctionSearch"]["pagedResults"]["results"][0]["auction"]["id"]
      aucioneer_name = result["data"]["auctionSearch"]["pagedResults"]["results"][0]["auction"]["auctioneer"]["name"]
      return auc_id, aucioneer_name
    else:
      return result
  else:
    print("Error:", response.status_code)
    return None
  

get_scheduled_auctions(92115)



(480682, 'Encore Auctions Inc')

In [ ]:
if __name__ == "__main__":
    args = sys.argv[1:]
    if len(args) == 1:
        aucioneer_id = args[0]
        while True:
            auc_id, aucioneer_name = get_scheduled_auctions(aucioneer_id, closest_auction_only=True)
            time_to_auction = get_lots_from_live_auction(auc_id, get_time_left=True)
            print(f"Next auction is in {time_to_auction} seconds")
            file_name = f"{aucioneer_name}_{auc_id}.json"
            print(file_name)
            
            # time.sleep(int(time_to_auction)-60)
            # track_and_update_data(auc_id, file_name, sleep_time=60)